# 2 Byte-Pair Encoding(BPE) Tokenizer

## 2.1 Unicode标准

在Python中，可以使用ord()方法将一个单独的Unicode字符转换为它的整数表示；可以使用chr()方法将一个整数Unicode码转换为其对应的字符（字符串类型）

In [43]:
ord("你")

20320

In [44]:
chr(20320)

'你'

### 问题 (unicode1): 理解Unicode

#### (a) Python语句chr(0)会返回什么Unicode字符？

In [35]:
chr(0)

'\x00'

In [36]:
ord("\x00")

0

答：chr(0)返回"\x00"，表示空字符

#### (b) 这个字符的字符串表示（\_\_repr\_\_()）与它的打印表示有什么不同？

In [37]:
s = "\x00"
print(f"字符{repr(s)}的字符串表示：")
print(f"s.__repr__():{s.__repr__()}")
print(f"repr(s):{repr(s)}")
print(f"字符{repr(s)}的打印表示：")
print(f"s.__str__():{s.__str__()}")
print(f"str(s):{str(s)}")
print(f"s:{s}")


字符'\x00'的字符串表示：
s.__repr__():'\x00'
repr(s):'\x00'
字符'\x00'的打印表示：
s.__str__(): 
str(s): 
s: 


答：该字符的字符串表示是未经转义的原始的字符，而其打印表示是经转义后的空字符

#### (c) 当这个字符出现在文本里会发生什么？在你的Python解释器中运行以下的语句可能会有所帮助，看看是否与你的预期相符
* chr(0)
* print(chr(0))
* "this is a test" + chr(0) + "string"
* print("this is a test" + chr(0) + "string")
* Deliverable: A one-sentence response.


In [38]:
chr(0)

'\x00'

In [39]:
print(chr(0))

 


In [40]:
"this is a test" + chr(0) + "string"

'this is a test\x00string'

In [41]:
print("this is a test" + chr(0) + "string")

this is a test string


答：在print函数中，该字符会被转义为空字符，不显示。

## 2.2 Unicode编码

为了将Unicode字符串编码为UTF-8，我们可以使用Python中的encode()方法，为了获取一个Python bytes对象的潜在字节值，我们可以迭代它（例如，调用list()），最后，我们可以使用decode()方法来解码一个UTF-8字节字符串为一个Unicode字符串。

In [46]:
test_string = "hello! こんにちは!"
test_string

'hello! こんにちは!'

In [47]:
utf8_encoded = test_string.encode("utf-8")
utf8_encoded

b'hello! \xe3\x81\x93\xe3\x82\x93\xe3\x81\xab\xe3\x81\xa1\xe3\x81\xaf!'

In [50]:
print(type(utf8_encoded))

<class 'bytes'>


得到编码字符串的字节值（从0到255的整数）

In [51]:
list(utf8_encoded)

[104,
 101,
 108,
 108,
 111,
 33,
 32,
 227,
 129,
 147,
 227,
 130,
 147,
 227,
 129,
 171,
 227,
 129,
 161,
 227,
 129,
 175,
 33]

很显然，一个字节值不一定对应一个Unicode字符

In [52]:
len(test_string)

13

In [53]:
len(utf8_encoded)

23

直接映射可能不会得到正确的结果

In [57]:
for i in list(utf8_encoded):
    print(f"{i}:{chr(i)}")

104:h
101:e
108:l
108:l
111:o
33:!
32: 
227:ã
129:
147:
227:ã
130:
147:
227:ã
129:
171:«
227:ã
129:
161:¡
227:ã
129:
175:¯
33:!


In [61]:
ord("こ")

12371

In [59]:
utf8_encoded.decode("utf-8")

'hello! こんにちは!'

把Unicode字符串转换为一个UTF-8字节序列，本质上就是把一个Unicode整数序列（范围是0\~154997）转换为另一个UTF-8整数序列（范围是0到255）。这说明任何输入文本（因为Unicode本质上已经算涵盖了世界上所有的符号）都可以被表示成由整数0\~255组成的序列。

### 问题（unicode2）：Unicode编码

#### (a)为什么在训练我们的分词器时，更倾向于使用UTF-8编码的字节，而不是UTF-16或UTF-32？比较这些编码对不同输入字符串的输出可能会有所帮助。

In [65]:
test_string = "hello! こんにちは! 你好！"
utf8_encoded = test_string.encode("utf-8")
utf8_encoded

b'hello! \xe3\x81\x93\xe3\x82\x93\xe3\x81\xab\xe3\x81\xa1\xe3\x81\xaf! \xe4\xbd\xa0\xe5\xa5\xbd\xef\xbc\x81'

In [68]:
len(utf8_encoded)

33

In [66]:
utf16_encoded = test_string.encode("utf-16")
utf16_encoded

b'\xff\xfeh\x00e\x00l\x00l\x00o\x00!\x00 \x00S0\x930k0a0o0!\x00 \x00`O}Y\x01\xff'

In [69]:
len(utf16_encoded)

36

In [67]:
utf32_encoded = test_string.encode("utf-32")
utf32_encoded

b'\xff\xfe\x00\x00h\x00\x00\x00e\x00\x00\x00l\x00\x00\x00l\x00\x00\x00o\x00\x00\x00!\x00\x00\x00 \x00\x00\x00S0\x00\x00\x930\x00\x00k0\x00\x00a0\x00\x00o0\x00\x00!\x00\x00\x00 \x00\x00\x00`O\x00\x00}Y\x00\x00\x01\xff\x00\x00'

In [70]:
len(utf32_encoded)

72

答：相较于UTF-16和UTF-32，UTF-8能将字符串编码为更短的字节序列，节省空间，同时有能力编码世界上绝大部分的文本。

#### (b)考虑以下（不正确的）函数，该函数旨在将UTF-8字节字符串解码为Unicode字符串。为什么这个函数不正确？请提供一个输入字节字符串的示例，该字符串会产生不正确的结果。

In [71]:
def decode_utf8_bytes_to_str_wrong(bytestring: bytes):
    return "".join([bytes([b]).decode("utf-8") for b in bytestring])

In [77]:
sample_encoded = "こんにちは".encode("utf-8")
decode_utf8_bytes_to_str_wrong(sample_encoded)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe3 in position 0: unexpected end of data

答：因为从Unicode字符映射到UTF-8字节并不一定是1对1的关系，函数只考虑一个字节解码为1个字符的情况了（当然，这对于常见的英文或者数字等可能是正确的）

#### 给出一个无法解码为任何Unicode字符的两字节序列。

In [79]:
byte_seq = bytes([0xe3,0xe3])

In [80]:
byte_seq.decode("utf-8")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe3 in position 0: invalid continuation byte

答：[0xe3,0xe3]

## 2.3 子词编解码(子词token化)

尽管字节级编解码解决了单词级编解码的out-of-vocabulary问题，但字节集编解码将文本转换成字节串会产生一个极长的输入序列，这会导致训练缓慢，长序列也会产生数据的长期依赖。

子词编解码(Subword Tokenization)是字节编解码和单词编解码的一种折中。例如，一个字节序列b'the'经常在文本数据中出现，则给它在词典中安排一个1字节的序列而不是3字节

这种思想产生了BPE Tokenization

## 2.4 BPE编码训练

BPE编解码器训练分为3个步骤：


1. 词典初始化：Tokenizer 词表是一个从字节字符串token到整数ID的一对一映射。由于我们要训练一个字节级的BPE tokenizer，我们初始的词典是一个所有字节的集合。由于有256种可能的字节值，所以初始词典大小是256.

2. 预编解码：为了避免数词频时的一些麻烦，我们需要预编解码语料库。你可以把这看作是语料库上的粗粒度标记化，它可以帮助我们计算成对字符出现的频率。例如，单词"text"出现了10次，当我们想统计t和e相邻的次数是多少时，由于我们知道text有t和e相邻，故可以直接把t和e相邻的次数加10。由于我们训练的是字节级BPE，每个token都被表示为一个UTF-8字节序列。

原始的BPE实现采用的预编解码手段是简单的空格分割（即s.split('')）。相反，这里我们将采用GPT-2使用的一种基于正则的预编解码器。from github.com/openai/tiktoken/pull/234/files :

PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""

使用此预标记器交互式分割一些文本以更好地了解其行为可能很有用：

In [81]:
!pip install regex

In [82]:
import regex as re
PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
re.findall(PAT, "some text that i'll pre-tokenize")

['some', ' text', ' that', ' i', "'ll", ' pre', '-', 'tokenize']

当实际使用时，当构建从预token到它们计数的映射时，应当使用re.finditer以避免保存预编解码的单词

In [93]:
it = re.finditer(PAT, "some text that i'll pre-tokenize")
list(it)

[<regex.Match object; span=(0, 4), match='some'>,
 <regex.Match object; span=(4, 9), match=' text'>,
 <regex.Match object; span=(9, 14), match=' that'>,
 <regex.Match object; span=(14, 16), match=' i'>,
 <regex.Match object; span=(16, 19), match="'ll">,
 <regex.Match object; span=(19, 23), match=' pre'>,
 <regex.Match object; span=(23, 24), match='-'>,
 <regex.Match object; span=(24, 32), match='tokenize'>]

3. 计算BPE合并：在高层次，BPE算法迭代地计数每一对字节，并识别频率最高的字节对（如"A", "B"）,每一次这一对词频最高的字节对出现，则合并它们（即用一个新的token"AB"替换它们），这一个新的token加入到词典中（例如，本来的词典是0到255，这个新的token可以是256）。最终，经过BPE训练后的词典大小为“**初始词典大小（256）+ BPE合并操作的执行次数 + 特殊tokens的数量**” 

为了提高BPE训练的效率，我们不考虑跨越预令牌边界的配对。当计算合并时，通过优先选择字典上较大的配对来确定配对频率中的联系。例如:如果字节对 (“A”, “B”), (“A”, “C”), (“B”, “ZZ”), 和 (“BA”, “A”) 都有最高的频率, 我们将合并 (“BA”, “A”):

In [94]:
max([("A", "B"), ("A", "C"), ("B", "ZZ"), ("BA", "A")])

('BA', 'A')

特殊tokens：有时会加入一些特殊tokens来编码元数据（例如<|endoftext|>），这些字符不会被分解为多个tokens，并且要被加入到词典中，有固定的tokenID

### 示例（bpe_example）:BPE训练示例

以下是一个原始论文中的BPE训练示例。

假设语料库中有如下文本：
low low low low low  
lower lower widest widest widest  
newest newest newest newest newest newest  
词典有特殊token:<|endoftext|>

词典：使用特殊token<|endoftext|>和256个字节值初始化词典

预编解码：为了简化，假设使用根据空格分割的预编解码。  
预编解码和计数后，得到频率表：  
{low: 5, lower: 2, widest: 3, newest: 6}

很容易把这个表表示为一个字典形式：dict[tuple[bytes], int],例如：{(l,o,w):5, {(l,o,w,e,r):2},...}。  
注意：在Python中，即使是单个字节也是bytes类型，在Python中没有byte类型来表示单个字节，正如没有char类型来表示单个字符。

合并：  
我们首先看每一个连续的字节对，并对它们出现的次数求和： {lo: 7, ow: 7, we: 8, er: 2, wi: 3, id: 3, de: 3, es: 9, st: 9, ne: 6, ew: 6}，字节对“es”和“st”都是最多的，并且平手了。于是我们选择字典序更大的“st”。合并预tokens中的st，得到： {(l,o,w): 5, (l,o,w,e,r): 2, (w,i,d,e,st): 3, (n,e,w,e,st): 6}  

在第二轮，同理得到（e,st）是词频最高的字节对（出现9次），把他们合并得到： {(l,o,w): 5, (l,o,w,e,r): 2, (w,i,d,est): 3, (n,e,w,est): 6}  
继续这个过程，最终我们会得到的合并序列是：['s t', 'e st', 'o w', 'l ow', 'w est', 'n e','ne west', 'w i', 'wi d', 'wid est', 'low e', 'lowe r']  

如果我们采用6个合并，我们有 ['s t', 'e st', 'o w', 'l ow', 'w est', 'n e']   
我们的词典最终会变成： [<|endoftext|>, [...256 BYTE CHARS], st, est, ow, low, west, ne]  
有了这个词典和合并集合，单词“newest”会被编码成[ne,west]

## 2.5 BPE tokenizer训练实验

在TinyStories数据集上训练一个字节级BPE tokenizer。

### 小提示

预分词并行化 你会发现，预分词步骤是一个主要瓶颈。你可以通过使用内置库multiprocessing对代码进行并行化来加速预分词。具体来说，我们建议在预分词的并行实现中，对语料库进行分块，同时确保分块边界出现在特殊标记的开头。你可以直接使用以下链接中的示例代码来获取分块边界，然后利用这些边界将工作分配到各个进程中：  
https://github.com/stanford-cs336/assignment1-basics/blob/main/cs336_basics/pretokenization_example.py  
这种分块将始终有效，因为我们永远不想跨文档边界合并。为了作业的目的，你总是可以这样拆分。不要担心接收不包含<|endoftext|>的非常大的语料库的边缘情况。

删除所有特殊标记 在使用正则表达式模式（使用re.finditer）运行预标记化之前，您应该从语料库（或块，如果使用并行实现）中删除所有特殊标记。确保在特殊标记上进行拆分，这样它们所分隔的文本之间就不会发生合并。例如，如果你有一个像[Doc 1]<|endoftext|>[Doc 2]这样的语料库（或块），你应该在特殊标记<|endoptext|>上拆分，并分别对[Doc 1]和[Doc 2]进行预标记，这样就不会在文档边界上发生合并。这可以通过使用re.split，并使用"|".join（special_tokens）作为分隔符来完成（请谨慎使用re.escape，因为|可能出现在特殊标记中）。test_train_bpe_special_token将对此进行测试。

优化合并步骤  上述程式化示例中BPE训练的天真实现很慢，因为对于每次合并，它都会迭代所有字节对以识别最频繁的对。但是，每次合并后唯一发生变化的对计数是与合并对重叠的计数。因此，通过索引所有对的计数并逐步更新这些计数，而不是显式迭代每对字节来计算对频率，可以提高BPE训练速度。使用此缓存过程可以显著提高速度，尽管我们注意到BPE训练的合并部分在Python中是不可并行的。

### 问题（train_bpe）: BPE Tokenizer Training

写一个函数，给定一个输入文本文件的地址，训练一个字节级BPE tokenizer。该BPE训练函数至少应包含以下输入参数：  
* input_path : str 指向一个有BPE tokenizer训练数据文件的路径
* vocab_size : int 一个正整数，定义最终词典的最大尺寸（包括初始字节词典，由合并过程中产生的词典项和任何特殊tokens）
* special_tokens : list[str] 加入到词典中的一个字符串列表。这些特殊tokens不影响BPE训练  
你的BPE训练函数应当返回结果词典和合并集：
* vocab : dict[int, bytes] 编解码器词典，一个从int（词典中的token ID）到bytes（token bytes）的映射
* merges: list[tuple[bytes, bytes]] 一个在BPE训练中产生的合并的列表。每个列表项都是一个bytes的元组（\<token1\>,\<token2\>），表示\<token1\>被合并到\<token2\>中。合并应当按照创建的顺序保存。

答：见bpe_train.py

### 问题（train_bpe_tinystories）: 在TinyStories数据集上训练BPE

#### (a)在TinStock数据集上训练字节级BPE标记器，最大词汇量为10000。  
请确保将TinyStories的特殊标记添加到词汇表中。序列化生成的词汇表并合并到磁盘以供进一步检查。
训练需要多少小时和内存？词汇表中最长的标记是什么？这有道理吗？

资源限制： ≤ 30 minutes (no GPUs), ≤ 30GB RAM

提示：应当能够在预编解码的过程中使用multiprocessing进行BPE训练的情况下，得到两分钟以下的时间。并得到如下事实：  
* <|endoftext|>标记用于分隔数据文件中的文档
* 在应用BPE合并之前，将<|endoftext|>标记作为特例处理



答：代码和执行见BPETrain.ipynb，执行时间为993.8秒

#### (b) 分析你的代码。tokenizer训练过程的哪个部分花费的时间最多？

### 问题（train_bpe_expts_owt）: 在OpenWebText数据集上训练BPE

#### (a) 在OpenWebText数据集上训练字节级BPE标记器，最大词汇量为32000。
序列化生成的词汇表并合并到磁盘以供进一步检查。词汇表中最长的标记是什么？这有道理吗？

资源限制: ≤ 12 hours (no GPUs), ≤ 100GB RAM

#### (b) 比较和对比您在TinyStories 和OpenWebText上训练得到的Tokenizer

## 2.6 BPE Tokenizer:编码和解码

我们之前实现了一个BPE tokenizer，可以取得tokenizer词汇表和一个merges列表，接下来我们将实现一个可以加载给定词汇表和一列merges的BPE tokenizer，并使用它们来编码（从文本到Token IDs）和解码（从Token Ids到文本）

### 2.6.1 编码文本

#### Step 1：预编码  
就像在训练BPE时做的那样预编码。然后，我们将在每个预token内部将这些字节合并为词汇表中的元素，独立处理每个预token

#### Step 2：应用merges  
采用由BPE训练期间产生的merges，把它应用到我们的预token上，以和训练时创造的merges相同的顺序进行

#### 示例：(bpe_encoding): BPE Encoding Example

例如，假设我们的输入字符串是'the cat ate'，我们的词汇表是：  
                                                {  
                                                0: b' ',   
                                                1: b'a',   
                                                2: b'c',   
                                                3: b'e',   
                                                4: b'h',   
                                                5: b't',   
                                                6: b'th',   
                                                7: b' c',   
                                                8: b' a',    
                                                9: b'the',   
                                                10: b'at'  
                                                }  
我们学到的merges是：  
[  
(b't', b'h'),   
(b' ', b'c'),   
(b' ', 'a'),   
(b'th', b'e'),  
(b' a', b't')  
]  
首先，我们的预编码器会将字符串拆分为['the', ' cat', ' ate']。  
然后，我们会逐一看每一个预token，并应用BPE merges  
  
第一个预token"the" 被首先表示为[b't',b'h',b'e']。查看我们的merges列表，我们发现第一个可应用的merge是(b't', b'h'), 所以应用它来转换我们的预token为[b'th',b'e']。  
然后，继续回到merges列表查找下一个可应用的merge是(b'th', b'e')，将预token转换为[b'the']。  
最后，我们发现merges列表中已经没有能应用于字符串的了(因为整个预token已经被合并成为单独的token)，至此我们已经应用了BPE merging，对应的整数序列是9

对剩下的预tokens继续应用BPE merging，我们看到' cat'被表示成 [b' c', b'a', b't'] 成为整数序列[7,1,5]。最后一个预token' ate'是[b' at', b'e']，整数序列[10,3]。  

因此，我们最终的编码结果是 [9, 7, 1, 5, 10, 3]

#### 特殊tokens  
你实现的tokenizer应当能够正确处理用户定义的特殊tokens（当构造tokenizer时提供）

#### 存储考虑  
假设我们想要对一个无法完全装入内存的大型文本文件进行分词。为了高效地处理这个大文件（或任何其他数据流），我们需要将其分解为可管理的块，并依次处理每个块，这样内存复杂度就是常量，而不是随文本大小线性增长。在这样做时，我们需要确保词元不会跨越块的边界，否则我们会得到与在内存中对整个序列进行分词的简单方法不同的分词结果。

### 2.6.2 解码文本  


为了把一串整数token IDs序列解码为原文本，我们可以简单查找每一个ID对应的条目（一个字节序列bytes），把它们拼接到一起，然后把bytes解码为Unicode字符串。  
注意，有的输入IDs不能保证映射为一个有效的Unicode字符串（由于用户可以输入任意的整数序列）。在这种情况下输入token IDs不能产生一个Unicode字符串，应当替换为官方的Unicode替换字符U+FFFD来替换这些格式错误的字节。bytes.decode方法中的error参数控制Unicode解码错误如何处理。使用errors='replace'将会自动替换为替换标识。

### 问题(tokenizer) :实现tokenizer

实现一个Tokenizer类，该类在给定词汇表和合并列表的情况下，将文本编码为整数ID，并将整数ID解码为文本。你的分词器还应支持用户提供的特殊词元（如果它们尚未存在于词汇表中，则将其追加到词汇表中）。我们建议使用以下接口：

def \_\_init\_\_(self, vocab, merges, special_tokens=None): 根据给定的词汇表、merges列表和（可选的）特殊tokens构建一个tokenizer。这个方法应当接收以下参数：
* vocab: dict[int, bytes]
* merges: list[tuple[bytes, bytes]]
* special_tokens: list[str] | None = None
  
def from_files(cls, vocab_filepath, merges_filepath, special_tokens=None):从一个序列化的词汇表和一个merges列表以及（可选的）特殊tokens构建并返回一个Tokenizer的类方法（和你的BPE训练器代码输出有相同的形式）。这个方法应当接收以下参数：  
* vocab_filepath: str
* merges_filepath: str
* special_tokens: list[str] | None = None

def encode(self, text: str) -> list[int]: 将一个输入文本编码为token IDs的序列。  

def encode_iterable(self, iterable: Iterable[str]) -> Iterator[int]: 给定一个字符串的可迭代对象（例如一个Python的文件handle），返回一个惰性生成token ID 的生成器。这对于我们无法直接加载到内存中的大文件进行内存高效分词是必需的。  

def decode(self, ids: list[int]) -> str: 将一个token IDs序列解码为文本  







答：见tokenizer.py

## 2.7 实验

### 问题（tokenizer_experiments）:tokenizer实验  
(a) 从TinyStories和OpenWebText中各采样10个文档。使用你之前训练的TinyStories和OpenWebText分词器（词汇表大小分别为10K和32K），将这些采样文档编码为整数ID。每个分词器的压缩比（字节/词元）是多少？  
  
(b) 如果你使用TinyStories分词器来分词OpenWebText样本会发生什么？比较压缩比和/或定性地描述会发生什么。  
  
(c) 估算你的分词器的吞吐量（例如，以字节/秒为单位）。分词Pile数据集（825GB文本）需要多长时间？  
  
(d) 使用你的TinyStories和OpenWebText分词器，将各自的训练和开发数据集编码为整数词元ID序列。我们稍后将使用这个来训练我们的语言模型。我们建议将词元ID序列化为uint16数据类型的NumPy数组。为什么uint16是一个合适的选择？  